In [2]:
import os, sys
import numpy as np
import pandas as pd

things_root = '/user_data/mmhender/stimuli/things/'
project_root = '/user_data/mmhender/featsynth/'
stimuli_folder = 'images_v1'

# this expt is all basic-level categorization

sys.path.append(project_root)

from code_utils import dropbox_utils, expt_utils
token_path = os.path.join(project_root, 'tokens/dbx_token.txt')
dbx = dropbox_utils.init_dropbox(token_path)

expt_name = 'expt3'


In [92]:
ti = pd.read_csv('/user_data/mmhender/featsynth/expt_design/expt3/trial_info_counterbal1_randorder0.csv')

ti.iloc[50:70]


,trial_num_overall,categ_ind,concept_ind,super_name,basic_name,ex_num,image_type_num,image_type,image_name,dropbox_url,target_present,cue_level_num,cue_level,cue_name,repeat_num,run_number,trial_in_run,random_order_number
50,925.0,4.0,6.0,vehicle,jeep,5.0,1.0,pool1,jeep_06s/grid5_1x1_upto_pool1.png,NaN,True,0.0,basic,jeep,1.0,1,51,0
51,270.0,1.0,3.0,bird,ostrich,0.0,2.0,pool2,ostrich_01b/grid5_1x1_upto_pool2.png,NaN,False,0.0,basic,duck,2.0,1,52,0
52,138.0,0.0,6.0,fruit,plum,8.0,4.0,pool4,plum_09s/grid5_1x1_upto_pool4.png,NaN,False,0.0,basic,kiwi,2.0,1,53,0
53,309.0,1.0,5.0,bird,peacock,9.0,2.0,pool2,peacock_10s/grid5_1x1_upto_pool2.png,NaN,True,0.0,basic,peacock,1.0,1,54,0
54,911.0,4.0,5.0,vehicle,golf_cart,1.0,2.0,pool2,golf_cart_02s/grid5_1x1_upto_pool2.png,NaN,True,0.0,basic,golf_cart,3.0,1,55,0
55,378.0,1.0,8.0,bird,toucan,8.0,4.0,pool4,toucan_09s/grid5_1x1_upto_pool4.png,NaN,False,0.0,basic,ostrich,2.0,1,56,0
56,411.0,2.0,0.0,home_decor,candelabra,1.0,2.0,pool2,candelabra_02s/grid5_1x1_upto_pool2.png,NaN,True,0.0,basic,candelabra,3.0,1,57,0
57,627.0,3.0,1.0,weapon,brass_knuckles,7.0,1.0,pool1,brass_knuckles_08s/grid5_1x1_upto_pool1.png,NaN,True,0.0,basic,brass_knuckles,3.0,1,58,0
58,577.0,2.0,8.0,home_decor,vase,7.0,4.0,pool4,vase_08s/grid5_1x1_upto_pool4.png,NaN,True,0.0,basic,vase,1.0,1,59,0
59,212.0,1.0,0.0,bird,chick,2.0,3.0,pool3,chick_03s/grid5_1x1_upto_pool3.png,NaN,False,0.0,basic,pigeon,0.0,1,60,0


In [5]:

def choose_image_subsets():
    
    # loading same set of categories used in expt1
    # but going to split them into four subsets here
    # these files are both made in /code_analysis/analyze_expt1.py
  
    fn2load = os.path.join(project_root, 'make_expt_designs', 'expt1_categ_info.npy')
    categ_info = np.load(fn2load, allow_pickle=True).item()
    
    fn2load = os.path.join(project_root, 'make_expt_designs', 'category_subsets_4sets.npy')
    groups = np.load(fn2load, allow_pickle=True)
    
    categ_sets = groups
    
    # figure out what the actual image filenames will be
    concepts_filename = os.path.join(things_root, 'concepts_use.npy')
    concepts_use = np.load(concepts_filename,allow_pickle=True).item()
    image_names = concepts_use['image_names']

    n_ex_use = 10;
    assert(np.mod(n_ex_use,10)==0)
    
    concept_sets = []
    image_name_sets = []
    for categ_set in categ_sets:

        cs = []
        ins = []

        for cat in categ_set:

            conc_names = categ_info['basic_names'][categ_info['super_names_long']==cat]
            cs += [conc_names]
            ins += [[image_names[conc][0:n_ex_use] for conc in conc_names]]

        concept_sets += [cs]
        image_name_sets += [ins]
    
    return categ_sets, concept_sets, image_name_sets


In [32]:
# decide which images we will use
categ_sets, concept_sets, image_name_sets = choose_image_subsets()
n_counterbalance_conds = len(categ_sets)

# figure out trial conditions/counts
n_categ_use = len(categ_sets[0])
n_concepts_use = len(concept_sets[0][0])
n_ex_use = len(image_name_sets[0][0][0])

# image types are original or "synth" from different DNN layers
n_layers=4
n_image_types = n_layers+1 

image_type_names = ['orig', 'pool1','pool2','pool3','pool4']

cue_levels = ['basic']
n_cue_levels = len(cue_levels); # basic or superordinate

n_repeats = 4
n_trials_per_concept = n_image_types * n_cue_levels * n_repeats

n_trials_total = n_categ_use * n_concepts_use * n_trials_per_concept

n_runs = 10;
assert(np.mod(n_trials_total, n_runs)==0)
n_trials_per_run = int(n_trials_total/n_runs);

rndseed = 232434
np.random.seed(rndseed)


In [30]:
cb = 0

print('making trial info for counterbalance cond %d of %d'%(cb, n_counterbalance_conds))

categ_use = categ_sets[cb]
print(categ_use)
concepts_use = concept_sets[cb]
image_names_use = image_name_sets[cb]

# make 100 different randomized orders for this same image set
trial_info_list = []
n_random_orders = 100;


making trial info for counterbalance cond 0 of 4
['fruit' 'bird' 'home_decor' 'weapon' 'vehicle']


In [69]:
rand = 0

trial_info = pd.DataFrame({'trial_num_overall': np.zeros((n_trials_total,)), 
                                       'categ_ind': np.zeros((n_trials_total,)),
                                      'concept_ind': np.zeros((n_trials_total,)),
                                      'super_name': np.zeros((n_trials_total,)),
                                      'basic_name': np.zeros((n_trials_total,)),
                                      'ex_num': np.zeros((n_trials_total,)),
                                      'image_type_num': np.zeros((n_trials_total,)),
                                      'image_type': np.zeros((n_trials_total,)),
                                      'image_name': np.zeros((n_trials_total,)),
                                      'dropbox_url': np.zeros((n_trials_total,)),
                                      'target_present': np.zeros((n_trials_total,)),
                                      'cue_level_num': np.zeros((n_trials_total,)),
                                      'cue_level': np.zeros((n_trials_total,)),
                                      'cue_name': np.zeros((n_trials_total,)),
                                      'repeat_num': np.zeros((n_trials_total,)),
                                      })



In [70]:
tt=-1
for ca in range(n_categ_use):

    for co in range(n_concepts_use):

        ex = -1

        for typ in range(n_image_types):
            
            for rep in range(n_repeats):

                tt+=1
                if np.mod(tt,100)==0:
                    print('proc trial %d of %d'%(tt, n_trials_total))

                trial_info['trial_num_overall'].iloc[tt] = tt
                trial_info['categ_ind'].iloc[tt] = ca
                trial_info['concept_ind'].iloc[tt] = co
                trial_info['super_name'].iloc[tt] = categ_use[ca]
                trial_info['basic_name'].iloc[tt] = concepts_use[ca][co]

                trial_info['image_type_num'].iloc[tt] = typ
                trial_info['image_type'].iloc[tt] = image_type_names[typ]
                trial_info['cue_level_num'].iloc[tt] = cue
                trial_info['cue_level'].iloc[tt] = cue_levels[cue]
                trial_info['repeat_num'].iloc[tt] = rep

                ex += 1
                ex = np.mod(ex, n_ex_use) # max of ex is 10
                
                trial_info['ex_num'].iloc[tt] = ex

                name_raw = image_names_use[ca][co][ex].split('.jpg')[0]

                if image_type_names[typ]=='orig':

                    trial_info['image_name'].iloc[tt] = os.path.join(name_raw, 'orig.png')
                    dbxpath = '/'+os.path.join(stimuli_folder, name_raw, 'orig.png')

                else:

                    trial_info['image_name'].iloc[tt] = os.path.join(name_raw, \
                                                                     'grid5_1x1_upto_%s.png'%\
                                                                     (image_type_names[typ]))
                    dbxpath = '/'+os.path.join(stimuli_folder,name_raw,\
                                           'grid5_1x1_upto_%s.png'%image_type_names[typ])

                # print(dbxpath)
                sys.stdout.flush()
                trial_info['dropbox_url'].iloc[tt] = ''
                # trial_info['dropbox_url'].iloc[tt] = dropbox_utils.get_shared_url(dbxpath, dbx)

                xx += 1;
                
                trial_info['target_present'].iloc[tt] = np.mod(rep, 2)==1
                # # determining the target present-absent based on which concept it is
                # # (this is not perfectly counter-balanced, but is balanced across all concepts)                
                # if np.mod(np.floor(xx/2),2)==0:
                #     trial_info['target_present'].iloc[tt] = (cue==0)
                # else:
                #     trial_info['target_present'].iloc[tt] = (cue==1)

            
# check that no duplicate images are being shown
assert(len(np.unique(trial_info['image_name']))==trial_info.shape[0])

proc trial 0 of 1000
proc trial 100 of 1000
proc trial 200 of 1000
proc trial 300 of 1000
proc trial 400 of 1000
proc trial 500 of 1000
proc trial 600 of 1000
proc trial 700 of 1000
proc trial 800 of 1000
proc trial 900 of 1000


In [61]:
1000/5/5

40.0

In [89]:
print('prepping cue names')
# decide what name to use to "cue" each trial, based on which level they are being cued at
# (these are all "correct names"; some of them get changed to incorrect in the next step)
trial_info['cue_name'] = trial_info['basic_name']

# Assign incorrect names to all the target-absent trials
# always swapping basic-level names across trials with same superordinate 
# level name, and same in all other attributes

for ca in range(n_categ_use):
    for typ in range(n_image_types):
        for rep in range(n_repeats):

            group = (trial_info['target_present']==False) & \
                (trial_info['categ_ind']==ca) & \
                (trial_info['image_type']==image_type_names[typ]) & \
                (trial_info['repeat_num']==rep)
            
            if np.sum(group)==0:
                # for some values of "repeat_num" there are no target-absent trials
                continue

            basic_name_list = np.array(concepts_use[ca])

            actual_basic_inds = np.array(trial_info['concept_ind'][group])
    
            shuff_basic_inds = expt_utils.shuffle_nosame(actual_basic_inds)

            shuff_basic_names = basic_name_list[shuff_basic_inds.astype(int)]
            
            trial_info['cue_name'].iloc[group] = shuff_basic_names


prepping cue names


In [87]:
# finally, shuffle the order of everything together.
shuff_order = np.random.permutation(np.arange(n_trials_total))
trial_info = trial_info.iloc[shuff_order]

# finally, assign the run numbers for each trial (these are the only non-shuffled columns).
trial_info['run_number'] = np.repeat(np.arange(n_runs), n_trials_per_run) + 1
trial_info['trial_in_run'] = np.tile(np.arange(n_trials_per_run), [n_runs,]) + 1
trial_info['random_order_number'] = np.full(fill_value=rand, shape=[n_trials_per_run*n_runs,])

# double check everything
print('checking trial info')
check_trial_info(trial_info)

checking trial info


In [86]:
def check_trial_info(ti):

    # make sure that the target present/target absent trials line up correctly
    # (cue name should match actual name for target present, and should 
    # always mismatch for target absent)

    inds_check = (ti['cue_level']=='basic') & (ti['target_present']==True)
    assert(np.all(ti['cue_name'][inds_check]==ti['basic_name'][inds_check]))
    inds_check = (ti['cue_level']=='basic') & (ti['target_present']==False)
    assert(np.all(ti['cue_name'][inds_check]!=ti['basic_name'][inds_check]))

    # check to make sure individual attributes are distributed evenly across trials 
    n_trials_total = ti.shape[0]

    attr_check_even = ['super_name', 'basic_name', 'ex_num', 'image_type_num', 'target_present', 'cue_level', 'repeat_num']
    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr], return_counts=True)
        assert(np.all(counts==n_trials_total/len(un)))


    # check the counterbalancing over multiple attributes

    # there should be an equal number of trials in each of the combinations of these 
    # different attribute "levels". for example each combination of category/image type. 

    attr_balanced = [ti['categ_ind'], ti['concept_ind'], ti['image_type_num'], ti['cue_level'], ti['target_present']]
    attr_balanced_inds = np.array([np.unique(attr, return_inverse=True)[1] for attr in attr_balanced]).T

    n_levels_each = [len(np.unique(attr)) for attr in attr_balanced]
    n_combs_expected = np.prod(n_levels_each)
    n_repeats_expected = n_trials_total/n_combs_expected

    un_rows, counts = np.unique(attr_balanced_inds,axis=0, return_counts=True)

    assert(un_rows.shape[0]==n_combs_expected)
    assert(np.all(counts==n_repeats_expected))


In [85]:
counts

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [67]:
# double check everything
print('checking trial info')
check_trial_info(trial_info)

(array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2]))

In [71]:
inds = trial_info['basic_name']=='apple'
trial_info[inds]

,trial_num_overall,categ_ind,concept_ind,super_name,basic_name,ex_num,image_type_num,image_type,image_name,dropbox_url,target_present,cue_level_num,cue_level,cue_name,repeat_num
0,0.0,0.0,0.0,fruit,apple,0.0,0.0,orig,apple_01b/orig.png,,False,0.0,basic,0.0,0.0
1,1.0,0.0,0.0,fruit,apple,1.0,0.0,orig,apple_02s/orig.png,,True,0.0,basic,0.0,1.0
2,2.0,0.0,0.0,fruit,apple,2.0,0.0,orig,apple_03s/orig.png,,False,0.0,basic,0.0,2.0
3,3.0,0.0,0.0,fruit,apple,3.0,0.0,orig,apple_04s/orig.png,,True,0.0,basic,0.0,3.0
4,4.0,0.0,0.0,fruit,apple,4.0,1.0,pool1,apple_05s/grid5_1x1_upto_pool1.png,,False,0.0,basic,0.0,0.0
5,5.0,0.0,0.0,fruit,apple,5.0,1.0,pool1,apple_06s/grid5_1x1_upto_pool1.png,,True,0.0,basic,0.0,1.0
6,6.0,0.0,0.0,fruit,apple,6.0,1.0,pool1,apple_07s/grid5_1x1_upto_pool1.png,,False,0.0,basic,0.0,2.0
7,7.0,0.0,0.0,fruit,apple,7.0,1.0,pool1,apple_08s/grid5_1x1_upto_pool1.png,,True,0.0,basic,0.0,3.0
8,8.0,0.0,0.0,fruit,apple,8.0,2.0,pool2,apple_09s/grid5_1x1_upto_pool2.png,,False,0.0,basic,0.0,0.0
9,9.0,0.0,0.0,fruit,apple,9.0,2.0,pool2,apple_10s/grid5_1x1_upto_pool2.png,,True,0.0,basic,0.0,1.0


In [37]:
cue

0

In [36]:
trial_info

,trial_num_overall,categ_ind,concept_ind,super_name,basic_name,ex_num,image_type_num,image_type,image_name,dropbox_url,target_present,cue_level_num,cue_level,cue_name
0,0.0,0.0,0.0,fruit,apple,0.0,0.0,orig,apple_01b/orig.png,,True,0.0,basic,0.0
1,1.0,0.0,0.0,fruit,apple,1.0,1.0,pool1,apple_02s/grid5_1x1_upto_pool1.png,,True,0.0,basic,0.0
2,2.0,0.0,0.0,fruit,apple,2.0,2.0,pool2,apple_03s/grid5_1x1_upto_pool2.png,,False,0.0,basic,0.0
3,3.0,0.0,0.0,fruit,apple,3.0,3.0,pool3,apple_04s/grid5_1x1_upto_pool3.png,,False,0.0,basic,0.0
4,4.0,0.0,0.0,fruit,apple,4.0,4.0,pool4,apple_05s/grid5_1x1_upto_pool4.png,,True,0.0,basic,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
n_trials_per_run

100

In [21]:
n_trials_per_concept

20

In [19]:
n_categ_use, n_concepts_use

(5, 10)

In [23]:
1000/5/5/10

4.0

In [15]:
cue_levels

['basic']